# Import 2D material from JARVIS database

Import a structure from the NIST JARVIS database by its ID. Use data at <a href="https://figshare.com/articles/dataset/Monolayer_data_for_heterostructure/22344571" target="_blank">JARVIS-DFT</a> to find the ID of the structure you are interested in.

<h2 style="color:green">Usage</h2>

1. Set Input Parameters (e.g. `JARVIS_ID`) below or use the default value
1. Click "Run" > "Run All Cells" to run all cells
1. Wait for the run to complete (it should take ~30 sec first time to install packages). Scroll down to view cell results.

## Methodology

The following happens in the script below:

1. Install `express-py` alongside the necessary packages
2. Import the material from JARVIS database
3. Convert the material to ESSE format
4. Pass the material to the outside runtime

## 1. Set Parameters

In [ ]:
# Note: JVASP-670 is an entry for MoTe2 and JVASP-6838 is an entry for GaTe
JARVIS_IDS = [
    "JVASP-670",
    "JVASP-6838",
]

## 2. Install Packages

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("import_material_from_jarvis_db_entry.ipynb")

## 3. Data Processing

### 3.1. Get data and wrap it in dataframe

In [ ]:
import pandas as pd
from jarvis.db.figshare import data

dft_2d = data("dft_2d")
dft_3d = data("dft_3d")
dataframe_2d = pd.DataFrame(dft_2d)
dataframe_3d = pd.DataFrame(dft_3d)
dataframe = pd.concat([dataframe_2d, dataframe_3d])

### 3.2. Preview the dataframe

In [ ]:
dataframe

### 3.3. Filter data

In [ ]:
import json
from express import ExPrESS

dataframe_entries_for_jarvis_ids = dataframe[(dataframe["jid"].isin(JARVIS_IDS))]
dataframe_entries_as_json = dataframe_entries_for_jarvis_ids.to_json(orient="records")
jarvis_db_entries = json.loads(dataframe_entries_as_json)

### 3.4. Convert to ESSE format

In [ ]:
def convert_jarvis_entry_to_esse(jarvis_db_entry):
    jarvis_db_entry_json = json.dumps(jarvis_db_entry)
    kwargs = {
        "structure_string": jarvis_db_entry_json,
        "cell_type": "original",
        "structure_format": "jarvis-db-entry",
    }
    handler = ExPrESS("structure", **kwargs)
    esse = handler.property("material", **kwargs)
    poscar = handler.parser.jarvis_db_entry_json_to_poscar(jarvis_db_entry_json)
    return (esse, poscar)


esse_entries = list(map(lambda e: convert_jarvis_entry_to_esse(e)[0], jarvis_db_entries))
poscars = list(map(lambda e: convert_jarvis_entry_to_esse(e)[1], jarvis_db_entries))

### 3.5. Preview the data

In [ ]:
from utils.visualize import visualize_materials
from mat3ra.made.tools.convert import from_poscar
from mat3ra.made.material import Material

# Uncomment to see the JSON
# print(json.dumps(data, indent=4))

materials = [Material.create(from_poscar(poscar)) for poscar in poscars]

visualize_materials(materials)

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

set_materials(materials)